<a href="https://colab.research.google.com/github/costaljorge/DadosSinteticos/blob/main/ModeloDadosSintenticos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Dados Sinteticos

In [ ]:
# 2024 será um ano crucial para os dados sintéticos.
# Estima-se que 60% dos dados utilizados para o desenvolvimento de IA e análises
# serão gerados sinteticamente, tornando-se um recurso essencial para diversos setores.

import pandas as pd
import random

# Definindo os gêneros de filmes
generos = ['Terror', 'Romance', 'Documentário', 'Ação', 'Suspense', 'Anime', 'Ficção', 'Comédia', 'Drama']

# Inicializando o DataFrame
df = pd.DataFrame(columns=generos)

# Gerando os dados para 200 linhas
for _ in range(200):
    # Inicializando a lista de filmes assistidos
    filmes_assistidos = []

    # Escolhendo aleatoriamente dois filmes para assistir
    for _ in range(2):
        filme = random.choice(generos)
        filmes_assistidos.append(filme)

    # Preenchendo o DataFrame com 1 para os filmes assistidos e 0 para os demais
    linha = {genero: 1 if genero in filmes_assistidos else 0 for genero in generos}
    #df = df.append(linha, ignore_index=True)
    df.loc[len(df)] = linha

# Salvando o DataFrame em um arquivo Excel
df.to_excel('Filmes.xlsx', index=False)

print("Arquivo salvo com sucesso!")


## Resolucao do Dataset

In [ ]:
# Importando as bibliotecas necessárias
# O objetivo desse modelo e tratar um dataset Binario,
# Com aprendizado Semi supervisionado,
# AgglomerativeClustering e DecisionTreeClassifier
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import tree
import matplotlib.pyplot as plt
import pandas as pd

# Importando o Dataset
df = pd.read_excel('/content/Filmes.xlsx')

# Calculando a matriz de distância usando a distância de Hamming
# A distância de Hamming mede a diferença entre duas strings de mesmo tamanho,
# indicando o número de caracteres que diferem.
# É utilizada em aprendizado de máquina (agrupamento, busca de similaridade, avaliação de modelos).
distancia = pdist(df.values, metric='hamming')
matriz_distancia = squareform(distancia)

# Realizando o agrupamento hierárquico
cluster = AgglomerativeClustering(n_clusters=3, affinity='precomputed', linkage='average')
predicoes = cluster.fit_predict(matriz_distancia)

# Adicionando os rótulos dos clusters ao DataFrame
df['cluster'] = predicoes

# Dividindo os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(df.drop('cluster', axis=1), df['cluster'], test_size=0.2, random_state=42)

# Treinando o modelo de árvore de decisão
clf = DecisionTreeClassifier(random_state=42, max_depth=3)
clf.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = clf.predict(X_test)

# Calculando a acurácia
acuracia = accuracy_score(y_test, y_pred)

# Imprimindo a árvore de decisão
plt.figure(figsize=(12, 8))
plot_tree(clf, filled=True, feature_names=df.columns, class_names=True, proportion=True)
# Imprimindo a acurácia
print('Acurácia:', acuracia)